In [1]:
import random
import math
import timeit
from numpy import unique
from numpy import array
from numpy import mean
from numpy import std
from numpy import bincount
from numpy import nonzero
from numpy import any
from numpy import random as nprandom
import itertools
import copy
import time
import csv
import pandas as pd
import ast
from collections import deque

In [2]:
class Card():
    def __init__(self,value,suit):
        self.value = value
        self.suit = suit
    def show(self):
        print(self.value,"of",self.suit)

In [3]:
suits = ["Hearts","Clubs","Diamonds","Spades"]
values = range(2,15)

In [4]:
class Deck():
    def __init__(self):
        #self.redraw_deck = []
        self.cards = deque([Card(val,suit) for val in values for suit in suits])
        #self.cards = [Card(val,suit) for val in values for suit in suits]
        self.shuffle() #Manually shuffle new deck
    def shuffle(self):
        nprandom.shuffle(self.cards)
    def show(self):
        for card in self.cards:
            card.show()
    def draw(self):
        ret = self.cards.popleft()
        return ret
    def drawBot(self):
        ret = self.cards.pop()
        return ret
    def bottom(self,card):
        self.cards.append(card)
    def showTop(self,num=5):
        index = 0
        while index < num:
            self.cards[index].show()
            index+=1
    #def redrawn(self,card):
    #    self.redraw_deck.append(card)

In [5]:
class PlayerHand():
    #Initial call merely initializes an empty hand
    #User must then dictate the kind of hand to be given
    def __init__(self):
        self.hand = [] #Will be list of card objects
    def show(self):
        for card in self.hand:
            card.show()
    def redraw(self,deck,redraw_lst):
        #print(redraw_lst)
        #print(self.hand)
        #This will only work for 5 or less total players where placing back into the deck is stricly unncessary
        #'''
        for ind in redraw_lst:
            #print(ind)
            deck.bottom(self.hand[ind])
            self.hand[ind] = deck.draw()
        return self
        #'''
    def undraw(self,deck,redraw_lst):
        #'''
        #Search reversed redraw_lst putting card in hand on top of deck and returning bottom of deck to hand in that position
        for ind in reversed(redraw_lst):
            deck.cards.appendleft(self.hand[ind]) #Insert card in hand to top of deck
            self.hand[ind] = deck.drawBot() #Replace card in this hand position with deck bottom card
        return self
        #'''
    #Functions defining what hand we're giving this player
    #Either random or specified hand, with possibly specific values
    def giveRand(self,deck):
        self.hand = [deck.draw() for i in range(5)]
        return self
    def giveFlush(self,deck): #Could specify a suit but there is no best suit so no real competitive point
        hand_values = []
        suit = None
        done = False
        ret = 0
        while not done:
            card = deck.draw()
            if suit == None:
                suit = card.suit
                hand_values.append(card.value)
                self.hand.append(card)
            else:
                #Already got the suit
                if card.suit == suit and card.value not in hand_values:
                    #Then we may add it
                    hand_values.append(card.value)
                    self.hand.append(card)
                else:
                    deck.bottom(card)
            if len(hand_values) == 5:
                
                #Check if straight, if yes go again, if no then done
                hand_values.sort() #sorted lowest to highest
                straight = True
                for ind in range(4):
                    if hand_values[ind] != hand_values[ind+1] - 1: #If left != right -1
                        straight = False
                        break
                
                if straight:
                    #Remove a card, doesn't matter which
                    rem = random.randint(0,4)
                    bot = self.hand.pop(rem)
                    hand_values.pop(hand_values.index(bot.value))
                    deck.bottom(bot)
                    ret = 1
                else:
                    done = True
        return ret
    
    def giveSpecificNearFlush(self,deck,flush_val_lst,other_val_lst,highest_suit_count):
        #Val_lst is 5 specific values, assume allows a flush so no repeat values
        self.flush_value_lst = [val for val in flush_val_lst]
        self.other_value_lst = [val for val in other_val_lst]
        self.hand = [0,0,0,0,0]
        suit_counts = {"Diamonds":0,
                      "Clubs":0,
                      "Hearts":0,
                      "Spades":0}
        #Get the first card
        while 1:
            card = deck.draw()
            if card.value in self.flush_value_lst:
                card_count = 1
                high_suit = card.suit
                suit_counts[high_suit] += 1
                ind = self.flush_value_lst.index(card.value) #First index location of this value
                #print(ind,card.value,self.value_lst,self.hand)
                self.hand[ind] = card
                self.flush_value_lst[ind] = None
                break
            else:
                deck.bottom(card)

        while 0 in self.hand:
            #loop_count += 1
            card=deck.draw()
            if card.value in self.flush_value_lst and card.suit == high_suit and suit_counts[high_suit] < highest_suit_count:
                card_count += 1
                suit_counts[card.suit] += 1
                ind = self.flush_value_lst.index(card.value) #First index location of this value

                self.hand[ind] = card
                self.flush_value_lst[ind] = None
            elif card.value in self.other_value_lst and card.suit != high_suit and highest_suit_count - suit_counts[high_suit] < 5 - card_count and suit_counts[card.suit] < highest_suit_count - 1:
                card_count += 1
                suit_counts[card.suit] += 1
                ind = self.other_value_lst.index(card.value) #First index location of this value

                self.hand[ind+highest_suit_count] = card
                self.other_value_lst[ind] = None
            else:
                deck.bottom(card)
            
        #Sorting by value by suit
        #self.show()
        #flush_hand = self.hand[0:highest_suit_count]
        #other_hand = self.hand[highest_suit_count:]
        
        #aa,flush_hand = zip(*sorted(zip(flush_val_lst,flush_hand),reverse=True))
        #bb,other_hand = zip(*sorted(zip(other_val_lst,other_hand),reverse=True))
        
        #self.hand = list(flush_hand+other_hand)

        return self
    
    #Haven't added straight flush
    #So improbable to get normally that there is no expected strategic value to getting rid of
    #This incredibly lucky hand. Royal flush is pointless since you can't do any better.
    def giveSpecific(self,deck,val_lst):
        #Val_lst is 5 specific values, we're gonna ignore flushes for simplicity
        #done = False
        #while not done:
        self.value_lst = [val for val in val_lst]
        self.hand = [0,0,0,0,0]
        while 0 in self.hand:
            card=deck.draw()
            if card.value in self.value_lst:
                ind = self.value_lst.index(card.value) #First index location of this value
                #print(ind,card.value,self.value_lst,self.hand)
                self.hand[ind] = card
                self.value_lst[ind] = None
            else:
                deck.bottom(card)
        return self

    ##Deprecated in this version, sort value list instead and enforce card placement based on that when getting cards in hand
    def sort(self):
        values = [card.value for card in self.hand]
        #Get the unique values and count per value
        counts = bincount(values)
        uniques = nonzero(counts)[0]
        counts = counts[nonzero(counts)]
        #Sort them by counts starting with highest to lowest
        #Then sort count ties by highest to lowest of uniques -- useful for ties
        counts,uniques = zip(*sorted(zip(counts, uniques),reverse=True))
        
        sorted_hand = []
        for ind,count in enumerate(counts):
            added_count = 0
            search_ind = len(self.hand) - 1
            while added_count < count:
                if self.hand[search_ind].value == uniques[ind]:
                    sorted_hand.append(self.hand[search_ind])
                    self.hand.pop(search_ind)
                    added_count += 1
                search_ind -= 1
        self.hand = sorted_hand
        
        return counts,uniques
    #Determine the score quality of the hand and tiebreakers
    def handInfo(self):
        #If we can pass this from previously sorting, do that
        values = [card.value for card in self.hand]
        #Get the unique values and count per value
        counts = bincount(values)
        uniques = nonzero(counts)[0]
        counts = counts[nonzero(counts)]
        #Sort them by counts starting with highest to lowest
        #Then sort count ties by highest to lowest of uniques -- useful for ties
        counts,uniques = zip(*sorted(zip(counts, uniques),reverse=True))
        '''
        4 of a kind
        '''
        if 4 in counts:
            score = 8
        elif 3 in counts:
            '''
            3 of a kind
            '''
            if len(counts) == 3:
                #3 of same value and 2 randos, so not fullhouse
                score = 4
            elif len(counts) == 2:
                '''
                Fullhouse
                '''
                #3 of a kind and a pair --> Fullhouse
                score = 7

        elif 2 in counts:
            #Could be a pair, 2 pair
            '''
            2 pair
            '''
            if len(counts) == 3:
                #2,2,1
                score = 3
            elif len(counts) == 4:
                '''
                1 pair
                '''
                #2,1,1,1
                score = 2
        else:
            #Looking at all singles, check for straight, flush stuff then call it nothing
            '''
            Flush Check
            '''
            flush = True
            suits = []
            for card in self.hand:
                if card.suit not in suits:
                    suits.append(card.suit)
                if len(suits) > 1:
                    flush = False
                    break
            '''
            Straight Check
            '''
            straight = True
            #Due to prior sorting, should be from highest to lowest card
            #As we read along uniques
            #Therefore we can just look for any time where the difference between
            #Current and next value are not 1
            for i in range(4):
                if uniques[i] - uniques[i+1] != 1:
                    straight = False
                    break
            #Now determine type of hand
            if flush and straight:
                if uniques[0] == 14:
                    '''
                    Royal Flush
                    '''
                    score = 10
                    #Technically the highest possible hand and results in legitimate tie
                else:
                    '''
                    Straight Flush
                    '''
                    score = 9
                    #Just check highest card, if equal then tied, if not someone lost/won
            elif flush:
                '''
                Flush
                '''
                score = 6
                ties = uniques
            elif straight:
                '''
                Straight
                '''
                score = 5
                #Again, highest card in straight defines straight
            else:
                '''
                Nothing
                '''
                score = 1
        return score,uniques

In [6]:
def checkWin(player_info,AI_info):
    #Info is list [score,ties]
    #Returning boolean of result in player's perspective
    if player_info[0] > AI_info[0]:
        #Player won
        return True
    elif player_info[0] < AI_info[0]:
        return False
    else:
        #Tied
        if player_info[1] > AI_info[1]:
            return True
        else:
            return False

In [7]:
#HOLY SHIT MAKE THESE LISTS LIKE THIS ALL THE TIME

def AI_Redraw(playerHand):
    score,ties = playerHand.handInfo()
    #redraw_lst = []
    if score >= 5:
        redraw_lst = []
        #Assume it's not worthwhile to trade in a straight or higher hand
    if score == 4:
        #Three of kind
        #Strategy will be to attempt a fullhouse/ 4 of a kind by trading in both other cards
        #Ties = [Threekind, spare1,spare2]
        
        redraw_lst = [ind for ind in range(5) if playerHand.hand[ind].value == ties[1] or playerHand.hand[ind].value == ties[2]]
        '''
        for ind,card in enumerate(playerHand.hand):
            if card.value == ties[1] or card.value == ties[2]:
                redraw_lst.append(ind)
        '''
    if score == 3:
        #Two Pair
        #Strategy will be to attempt fullhouse
        redraw_lst = [ind for ind in range(5) if playerHand.hand[ind].value == ties[2]]
        '''
        for ind,card in enumerate(playerHand.hand):
            if card.value == ties[2]:
                redraw_lst.append(ind)
        '''
    if score == 2:
        #Pair
        #Keep pair, remove rest
        #Ties = [Pair, Spare1,Spare2,Spare3]
        redraw_lst = [ind for ind in range(5) if playerHand.hand[ind].value == ties[1] or playerHand.hand[ind].value == ties[2] or playerHand.hand[ind].value == ties[3]]
        '''
        for ind,card in enumerate(playerHand.hand):
            if card.value == ties[1] or card.value == ties[2] or card.value == ties[3]:
                redraw_lst.append(ind)
        '''
    if score == 1:
        #Singles
        #Keep highest, remove rest
        redraw_lst = [ind for ind in range(5) if not playerHand.hand[ind].value == ties[0]]
        '''
        for ind,card in enumerate(playerHand.hand):
            if not card.value == ties[0]:
                redraw_lst.append(ind)
        '''
    return redraw_lst

# Step through specific hand, all redraw varieties to find best decision

In [8]:
#All unique hand combinations
card_values = [2,3,4,5,6,7,8,9,10,11,12,13,14]
deck_values = []
for card_value in card_values:
    for i in range(4):
        deck_values.append(card_value)
hand_combs = list(itertools.combinations(deck_values,5))
hand_comb_uniques = unique(array(hand_combs),axis=0)
#Uniques ignoring royal flush, straight flush, flush
#Basically non-flush related ones
global redraw_combs
redraw_combs=[]
for i in range(6):
    test = list(itertools.combinations([0,1,2,3,4],i))
    for j in test:
        redraw_combs.append(j)
        
#Spend a bit more time making them into mutable lists rather than tuples for later usage

In [9]:
nearness = 4
final_uniques = []
for unique_hand in hand_comb_uniques:
    uniques,counts = unique(array(unique_hand),return_counts = True)
    if len(counts) >= nearness:
        final_uniques.append(list(unique_hand))

In [10]:
pair_count = 0
other_count = 0
for hand in final_uniques:
    uniques,counts = unique(array(hand),return_counts = True)
    if 2 in counts:
        #a pair
        pair_count += 1
    else:
        other_count += 1
print(pair_count,other_count)

2860 1287


# Game Simulations and CSV Writes

In [11]:
global simIndex
simIndex = 0

In [12]:
def finalSims(startind,stopind,total_games,AI_players,highest_suit_count,filename,write=True,verbose=True):
    for index in range(startind,stopind,+1):
        global simIndex
        simIndex = index
        if verbose:
            print("Processing Index:",index)
        #Game type value definitions
        games = total_games #150,000 Gives std of around 0.003 which I find acceptable, higher games count decreases but diminishing returns/time
        #Get counts,uniques in this set of values
        values = list(final_uniques[index])
        counts = bincount(values)
        uniques = nonzero(counts)[0]
        counts = counts[nonzero(counts)]
        counts,uniques = zip(*sorted(zip(counts, uniques),reverse=True))
        values = [uniques[ind] for ind,c in enumerate(counts) for i in range(c)]
        
        #Get set of value lists for being near a flush
        if any([array(counts) > 1]):
            value_combs = []
            non_flush_vals = [uniques[i] for i in range(len(counts)) if counts[i] > 1]
            lsts = itertools.combinations(list(uniques),5-len(non_flush_vals))
            for l in lsts:
                additionals = [v for v in uniques if v not in l]
                value_combs.append(list(l)+list(additionals)+list(non_flush_vals))
        else: #Only singles
            value_combs = [values[:i]+values[i+1:]+values[i:i+1] for i in range(5)]
        
        redraw_unique_combs = redraw_combs
        
        #Form player hand and determine what the unique ways of redrawing are
        #If we have duplicate values this cuts down runtime a lot and removes duplicate rows
        deck = Deck()
        ph = PlayerHand().giveSpecific(deck,values)
        #ph.sort()
        hand_score,ties = ph.handInfo() #Also grab this for later csv writing
        
        val_lst_index = 0
        for val_lst in value_combs:
            start = time.time()
            if verbose:
                print("Processing variation",val_lst)
            flush_val_lst = val_lst[0:highest_suit_count]
            other_val_lst = val_lst[highest_suit_count:]
            
            #Checker to see if we've done this already
            if write:
                df = pd.read_csv(filename, usecols=["Hand List"])
                completed_hands = df["Hand List"].tolist()
                completed_hands = list(unique(completed_hands))
                dummy = []
                for hand in completed_hands:
                    dummy.append(ast.literal_eval(hand))
                if val_lst in dummy:
                    #Checks to see if we've gone through all iterations for this hand before calling it off
                    if val_lst_index == len(value_combs) - 1:
                        raise ValueError("We already ran through this hand!!")
                    val_lst_index += 1
                    continue #bypasses all the data collection and moves on since we know this hand is done
            
            '''
            redraw_remove_value_sets = []
            redraw_unique_combs = []

            #Getting just unique redrawing
            for ind, redraw_ind_lst in enumerate(redraw_combs):
                remove = []
                for ind in redraw_ind_lst:
                    remove.append(val_lst[ind])
                if remove not in redraw_remove_value_sets:
                    redraw_remove_value_sets.append(remove)
                    redraw_unique_combs.append(redraw_ind_lst)
            #print(redraw_unique_combs)
            '''
            #Simulate games
            wins = [0] * len(redraw_unique_combs)

            for i in range(games):
                deck= Deck()
                playerHand = PlayerHand().giveSpecificNearFlush(deck,flush_val_lst,other_val_lst,highest_suit_count)
                #playerHand.show()
                #Then we haven't added any wins for this redraw type
                #Generate random hands for AI and determine the redrawing indices
                AI_Hands = [PlayerHand().giveRand(deck) for i in range(AI_players)]
                AI_redraw_lsts = [AI_Redraw(AI_Hands[i]) for i in range(AI_players)]
                ind = 0
                for redraw_lst in redraw_unique_combs:
                    #Redraw for the player
                    #Assumed sorted
                    playerHand.redraw(deck,redraw_lst)
                    #Redraw for AI
                    for player_ind in range(AI_players):
                        AI_Hands[player_ind].redraw(deck,AI_redraw_lsts[player_ind])
                    #Check if we lost or not
                    lost = True
                    for i in range(AI_players):
                        if checkWin(playerHand.handInfo(),AI_Hands[i].handInfo()):
                            lost = False
                            break
                    #Undo all of the card drawing to reset hands/deck
                    for i in range(AI_players-1,-1,-1):
                        AI_Hands[i].undraw(deck,AI_redraw_lsts[i])
                    playerHand.undraw(deck,redraw_lst)
                    if not lost:
                        wins[ind] += 1
                        #wins[redraw_unique_combs.index(redraw_lst)] += 1
                    ind += 1

            winrates = [count/games for count in wins]
            stds = [2.576*(((rate)-(rate)**2)/games)**0.5 for rate in winrates] #99% confidence interval


            #Sort important lists by winrate
            winrates,redraw_unique_combs,stds = zip(*sorted(zip(winrates,redraw_unique_combs,stds),reverse=True))

            #Info for csv file

            #print("Hand",val_lst)
            #print('')
            keep_inds = [[j for j in range(5) if j not in redraw_unique_combs[i]] for i in range(len(redraw_unique_combs))]
            remove_inds = [[j for j in range(5) if j in redraw_unique_combs[i]] for i in range(len(redraw_unique_combs))]
            if not write and verbose:
                for i in range(len(redraw_unique_combs)):
                    print("Winrate:",winrates[i])
                    print("Standard Dev:",stds[i])
                    print("Redraw removing:",remove_inds[i])
                    print("Keeping:",keep_inds[i])
                    print("-----------------------------")
                    
            #end = time.time()
            #print("Algorithm Time Elapsed:",end-start)

            end = time.time()
            print("Algorithm Time Elapsed:",end-start)
            if write:
                for i in range(len(redraw_unique_combs)):
                    f = open(filename,'a',newline='')
                    writer = csv.writer(f)
                    row = [val_lst,hand_score,keep_inds[i],remove_inds[i],winrates[i],stds[i]]
                    writer.writerow(row)
                    f.close()

In [ ]:
for i in range(1,len(final_uniques)):
    try:
        finalSims(i,i+1,100000,4,nearness,"Flush Data.csv",write=True,verbose=True) #Ends at 4147; 1287 nothings/straights and 2860 pairs; therefore final data should contain 5*1287+2860 hands = 9295 hands
        #Each hand then gets 32 different redrawing types so total rows of 297,440
    except:
        print("Hand done already")
#Probably need to cycle through every iteration once completed to clean up missing data

In [ ]:
final_final_uniques = []
for j in range(len(final_uniques)):
    values = list(final_uniques[j])
    counts = bincount(values)
    uniques = nonzero(counts)[0]
    counts = counts[nonzero(counts)]
    counts,uniques = zip(*sorted(zip(counts, uniques),reverse=True))
    values = [uniques[ind] for ind,c in enumerate(counts) for i in range(c)]

    #Get set of value lists for being near a flush
    if any([array(counts) > 1]):
        value_combs = []
        non_flush_vals = [uniques[i] for i in range(len(counts)) if counts[i] > 1]
        lsts = itertools.combinations(list(uniques),5-len(non_flush_vals))
        for l in lsts:
            additionals = [v for v in uniques if v not in l]
            value_combs.append(list(l)+list(additionals)+list(non_flush_vals))
    else: #Only singles
        value_combs = [values[:i]+values[i+1:]+values[i:i+1] for i in range(5)]
    
    for comb in value_combs:
        final_final_uniques.append(comb)

In [ ]:
#Form player hand and determine what the unique ways of redrawing are
#If we have duplicate values this cuts down runtime a lot and removes duplicate rows
deck = Deck()
ph = PlayerHand().giveSpecific(deck,values)
#ph.sort()
hand_score,ties = ph.handInfo() #Also grab this for later csv writing

In [46]:
final_final_uniques.index([13,12,10,6,14])

8710